<a href="https://colab.research.google.com/github/XueqingWu/Explainable_AI_Global_Explanation/blob/main/Global_Explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install shap==0.45.1
!pip install git+https://github.com/MaximeJumelle/ALEPython.git@dev#egg=alepython

  Cloning https://github.com/MaximeJumelle/ALEPython.git (to revision dev) to /tmp/pip-install-i601au1n/alepython_1231e4aade624a168ac644cb17d369e3
  Running command git clone --filter=blob:none --quiet https://github.com/MaximeJumelle/ALEPython.git /tmp/pip-install-i601au1n/alepython_1231e4aade624a168ac644cb17d369e3
  Resolved https://github.com/MaximeJumelle/ALEPython.git to commit 286350ab674980a32270db2a0b5ccca1380312a7
  Preparing metadata (setup.py) ... done


In [7]:

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree, DecisionTreeClassifier
from sklearn import metrics

# Models
import xgboost
from sklearn.model_selection import train_test_split

# XAI
import shap
from alepython import ale_plot
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import permutation_importance

np.random.seed(13)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [4]:

# load dataset
from sklearn.datasets import load_iris

iris = load_iris()
# convert to dataframe
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df
# # Subset data to two classes
iris_df = iris_df[iris.target != 2]
iris_df["target"] = iris.target[iris.target != 2]
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1
96,5.7,2.9,4.2,1.3,1
97,6.2,2.9,4.3,1.3,1
98,5.1,2.5,3.0,1.1,1


In [6]:
from sklearn.model_selection import train_test_split

# Assuming iris_df is your DataFrame and target is the column you want to predict
y = iris_df["target"].to_numpy()  # Target variable
X = iris_df.drop("target", axis=1)
features=X.columns
X=X.to_numpy()  # Features as numpy array

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now X_train, X_test, y_train, y_test can be used for training and testing

In [12]:
X_test

array([[6. , 2.7, 5.1, 1.6],
       [5.5, 2.3, 4. , 1.3],
       [5.9, 3.2, 4.8, 1.8],
       [4.8, 3. , 1.4, 0.3],
       [5.1, 3.8, 1.9, 0.4],
       [5.1, 3.4, 1.5, 0.2],
       [4.6, 3.6, 1. , 0.2],
       [5.5, 2.4, 3.8, 1.1],
       [5.4, 3.7, 1.5, 0.2],
       [5.1, 3.5, 1.4, 0.2],
       [5.7, 3.8, 1.7, 0.3],
       [4.8, 3.1, 1.6, 0.2],
       [6.1, 2.8, 4.7, 1.2],
       [5.5, 4.2, 1.4, 0.2],
       [5.5, 2.6, 4.4, 1.2],
       [5. , 3.6, 1.4, 0.2],
       [6.8, 2.8, 4.8, 1.4],
       [6.7, 3. , 5. , 1.7],
       [4.8, 3. , 1.4, 0.1],
       [5.4, 3.4, 1.5, 0.4],
       [5.7, 2.8, 4.5, 1.3],
       [5.6, 3. , 4.1, 1.3],
       [5. , 3.4, 1.6, 0.4],
       [4.4, 3.2, 1.3, 0.2],
       [5.6, 2.5, 3.9, 1.1],
       [5.7, 4.4, 1.5, 0.4],
       [5. , 3.5, 1.3, 0.3],
       [5.7, 2.9, 4.2, 1.3],
       [4.9, 3.1, 1.5, 0.1],
       [6.3, 2.5, 4.9, 1.5],
       [4.8, 3.4, 1.6, 0.2],
       [4.6, 3.2, 1.4, 0.2],
       [6. , 3.4, 4.5, 1.6],
       [5.2, 3.4, 1.4, 0.2],
       [5. , 2

## Train Model

In [19]:
# Train XGBoost model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
model = xgboost.XGBClassifier().fit(X_train, y_train)

In [15]:
X_test.iloc[:, 0]

array([6. , 2.7, 5.1, 1.6])

## Partial Dependence Plots (PDP)

In [20]:

# Choose the feature of interest
features =[:, 0]

# Use PartialDependenceDisplay to plot PDP
PartialDependenceDisplay.from_estimator(model, X_test, features, kind='average') #kind='both'

SyntaxError: invalid syntax (<ipython-input-20-b55431f3a316>, line 2)